### File Loads & Text Scraping

In [2]:
#Package imports
!pip install yfinance
from bs4 import BeautifulSoup
import pandas as pd
import time
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from googlesearch import search
import yfinance as yf #this requires a package adjustment to work (yikes)
sns.set_style('darkgrid')

In [4]:
companies_df = pd.read_csv("companies.csv")
companies_df

,Company (Name)
0,ABC Technologies
1,Agnico Eagle
2,Alamos Gold
3,Bird Construction Inc.
4,Blackberry
...,...
67,Travelers Insurance Company of Canada
68,Wajax Corporation
69,"Waste Connections, Inc."
70,Wesdome Gold


### Functions to Extract Tickers & Mkt Cap

In [29]:
#Function to convert company names to tickers
def ticker_convert(self):
    searchval = 'yahoo finance '+self
    link = []
    #limits to the first link
    for url in search(searchval, tld='es', lang='es', stop=1):
        link.append(url)
    link = str(link[0])
    link=link.split("/")
    if link[-1]=='':
        ticker=link[-2]
    else:
        x=link[-1].split('=')
        ticker=x[-1]
    return(ticker)

#Function converts arrays of names into arrays of tickers using name_convert
def ticker_array_convert(self):
    ticker_list = []
    for i in self:
        val = ticker_convert(i)
        ticker_list.append(val)
    return(ticker_list)

#Function uses yfinance package to pull forward various important values from yahoo finance
def company_info(self):
    company = yf.Ticker(self)
    info = company.info
    try: market_cap = info['marketCap']
    except:
      return ['N/A']
    return [market_cap]

#Function pulls company_data func to pull values from yahoo finance for an array of tickers
def company_info_df(self):
    data_array = []
    for i in self:
        val = company_info(i)
        data_array.append(val)
        info_df = pd.DataFrame(np.vstack(data_array),columns = ['MARKET_CAP'] )
    return(info_df)

### Analyzing List

In [6]:
companies_list = list(companies_df['Company (Name)'])
companies_tickers = ticker_array_convert(companies_list)

In [7]:
companies_tickers[0]

'ABCT.TO'

In [8]:
tickers_df = pd.DataFrame()
tickers_df['Name'] = companies_list
tickers_df['Ticker'] = companies_tickers
tickers_df

,Name,Ticker
0,ABC Technologies,ABCT.TO
1,Agnico Eagle,AEM
2,Alamos Gold,AGI
3,Bird Construction Inc.,BDT.TO
4,Blackberry,BB
...,...,...
67,Travelers Insurance Company of Canada,TRV
68,Wajax Corporation,WJX.TO
69,"Waste Connections, Inc.",WCN
70,Wesdome Gold,wdo.to


In [26]:
tickers_info = company_info_df(tickers_df['Ticker'])

### Exporting to Excel

In [37]:
export_df = pd.concat([tickers_df,tickers_info],axis=1)
export_df

,Name,Ticker,MARKET_CAP
0,ABC Technologies,ABCT.TO,803280960
1,Agnico Eagle,AEM,27465371648
2,Alamos Gold,AGI,3178543872
3,Bird Construction Inc.,BDT.TO,489701120
4,Blackberry,BB,4400095232
...,...,...,...
67,Travelers Insurance Company of Canada,TRV,45102723072
68,Wajax Corporation,WJX.TO,424599200
69,"Waste Connections, Inc.",WCN,35347259392
70,Wesdome Gold,wdo.to,2166706176


In [38]:
#Exporting
export_df.to_excel("company_data.xlsx",sheet_name='Sheet 1')  